In [1]:
from ctransformers import AutoModelForCausalLM
from pymongo import MongoClient
import datetime
import uuid
import chromadb
from sentence_transformers import SentenceTransformer
import logging

In [2]:
logging.getLogger("chromadb").setLevel(logging.CRITICAL)
logging.getLogger().setLevel(logging.CRITICAL)

In [3]:
# mongo db connection

try:
    client = MongoClient("mongodb://localhost:27017/")
    db = client["assistant_db"]
    users_collection = db["users"]
    chats_collection = db["chats"]
    print("Connected to MongoDB!")

except Exception as e:
    print(f"MongoDB Connection Failed: {e}")
    exit()

Connected to MongoDB!


In [4]:
# embedding model & chroma db

client_chroma = chromadb.PersistentClient(path="./chroma_db")
db_chroma = client_chroma.get_or_create_collection("chat_context")

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# load mistral 7b

llm = AutoModelForCausalLM.from_pretrained(
    "models/",
    model_file = "mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    model_type = "mistral",
    gpu_layers = 25,
    temperature = 0.1,
    top_k = 30,
    top_p = 0.85,
    max_new_tokens = 250,
    stream = False,
    seed = 42,
    context_length = 4096,
)

print("\nModel loaded successfully\n")



Model loaded successfully



In [6]:
user_id = None

def get_or_create_user():
    global user_id
    username = input("Enter your username: ").strip()

    user = users_collection.find_one({"username": username})

    if user:
        print("🔹 User found! Loading profile...\n")
        user_id = user["_id"]
    else:
        print("🔹 New user detected! Let's create your profile.\n")
        user_data = {
            "_id": str(uuid.uuid4()),
            "username": username,
            "age": int(input("Enter your age: ")),
            "gender": input("Identify yourself as [Male, Female, Others]: ").strip(),
            "location": input("Where are you from: ").strip(),
            "mother-tongue": input("Your mother tongue: ").strip(),
            "bio": input("Enter a short bio: ").strip(),
            "interests": input("Enter your interests (comma-separated): ").split(","),
            "preferred_topics": input("Topics you wish to explore [tech, health, stocks]: ").strip(),
        }
        user_id = user_data["_id"]
        users_collection.insert_one(user_data)
        print("Profile created successfully!\n")

get_or_create_user()

🔹 User found! Loading profile...



In [7]:
def store_chat_in_chroma(query, response, context):
    try:
        chat_id = str(uuid.uuid4())
        
        existing_ids = db_chroma.get(ids=[chat_id])
        if existing_ids and len(existing_ids["documents"]) > 0:
            return 

        response_vector = sentence_model.encode([response])[0]
        context_vector = sentence_model.encode([context])[0]

        db_chroma.add(
            ids=[chat_id],
            embeddings=[response_vector],
            documents=[response],
            metadatas=[{"context": context}],
        )
        
        db_chroma.add(
            ids=[chat_id],
            embeddings=[context_vector],
            documents=[context],
            metadatas=[{"context": context}],
        )

    except Exception as e:
        print(f"Failed to save chat in ChromaDB: {e}")


In [ ]:
def retrieve_context_from_chroma(query, top_k = 4):
    """Retrieve the most relevant context from ChromaDB"""
    query_vector = sentence_model.encode([query])[0]
    results = db_chroma.query(query_embeddings=[query_vector], n_results=top_k)
    
    context = ""

    for result in results['documents']:
        context += " ".join(result) + "\n" if isinstance(result, list) else result + "\n"
    
    return context


In [ ]:
while True:
    query = input("query: ").strip()

    if query.lower() in ["exit", "quit", "bye"]:
        print("🤖 Goodbye!")
        break

    print(f"User: {query}\n")

    user_data = users_collection.find_one({"_id": user_id})

    if not user_data:
        print("❌ User profile not found. Proceeding without context.")
        user_context = "No additional user information available."
    else:
        user_context = (
            f"User Name: {user_data.get('full_name', 'No full name available')}, "
            f"User Age: {user_data.get('age', 'No age available')}, "
            f"User Gender: {user_data.get('gender', 'No gender available')}, "
            f"User Location: {user_data.get('location', 'No location available')}, "
            f"User Bio: {user_data.get('bio', 'No bio available')}, "
            f"Interests: {', '.join(user_data.get('interests', []))}, "
            f"Preferred Topics: {', '.join(user_data.get('preferred_topics', []))}, "
            f"Favorite Books: {', '.join(user_data.get('favorite_books', []))}, "
            f"Favorite Movies: {', '.join(user_data.get('favorite_movies', []))}, "
            f"Occupation: {user_data.get('occupation', 'Unknown')}, "
            f"Skills: {', '.join(user_data.get('skills', []))}, "
            f"Learning Goals: {', '.join(user_data.get('learning_goals', []))}, "
            f"Timezone: {user_data.get('timezone', 'Unknown')}, "
            f"Preferred Response Tone: {user_data.get('preferred_response_tone', 'Neutral')}."
        )

    previous_context = retrieve_context_from_chroma(query)
    full_context = f"{user_context}\nPrevious Context:\n{previous_context}"

    prompt = f"""
    User: {query}\n
    Instruction: Only answer to the question, nothing extra. Only provide information if you have access to factual data, if you don't have the information, please say so explicitly.\n
    Context: {full_context}\n
    Assistant (Concise & Professional):
"""
    response = llm(prompt)

    print(f"Assistant: {response}\n")
    
    # store chat inside of mongodb
    try:
        chat_data = {
            "user_id": user_id,
            "query": query,
            "response": response,
            "context": full_context,
            "timestamp": datetime.datetime.now(datetime.UTC)
        }
        chats_collection.insert_one(chat_data)


        store_chat_in_chroma(query, response, full_context)

    except Exception as e:
        print(f"Failed to save chat: {e}")

User: what are my upcoming meetings

Predicted Contexts:

Assistant:     You have meetings coming up with Samantha Wheeler on the 8th of April at 4pm, and with Josh on the 6th of April at RSET.

🤖 Goodbye!
